In [1]:
import keras
from keras import regularizers
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Conv1D
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
import tensorflow as tf
from sklearn.metrics import cohen_kappa_score, accuracy_score
import numpy as np
%matplotlib inline
import os
from PIL import ImageFile,Image
ImageFile.LOAD_TRUNCATED_IMAGES = True
import scipy
import json
from keras.models import load_model
import gensim
from bs4 import BeautifulSoup
import requests
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import nltk
nltk.download('stopwords')
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gurne\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
main_model = load_model('model_85acc.h5')
ftt_model = gensim.models.Word2Vec.load("FastTest.kv")

In [34]:
link = input("Enter youtube link")

In [49]:
response = requests.request("GET", link)
soup = BeautifulSoup(response.text, "html.parser")
body = soup.find_all("body")[0]
scripts = body.find_all("script")
result = json.loads(scripts[0].string[30:-1])

In [108]:
soup2 = BeautifulSoup(response.content, "html.parser")
soup2.find_all("body")
soup2

<!DOCTYPE html>
<html darker-dark-theme="" darker-dark-theme-deprecate="" lang="en-GB" style="font-size: 10px;font-family: Roboto, Arial, sans-serif;" system-icons="" typography="" typography-spacing=""><head><script data-id="_gd" nonce="EdmiCMfg53MylDVD4abYhA">window.WIZ_global_data = {"MuJWjd":false,"nQyAE":{},"oxN3nb":{"1":false}};</script><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="ApvK67ociHgr2egd6c2ZjrfPuRs8BHcvSggogIOPQNH7GJ3cVlyJ1NOq/COCdj0+zxskqHt9HgLLETc8qqD+vwsAAABteyJvcmlnaW4iOiJodHRwczovL3lvdXR1YmUuY29tOjQ0MyIsImZlYXR1cmUiOiJQcml2YWN5U2FuZGJveEFkc0FQSXMiLCJleHBpcnkiOjE2OTUxNjc5OTksImlzU3ViZG9tYWluIjp0cnVlfQ==" http-equiv="origin-trial"/><script nonce="EdmiCMfg53MylDVD4abYhA">var ytcfg={d:function(){return window.yt&&yt.config_||ytcfg.data_||(ytcfg.data_={})},get:function(k,o){return k in ytcfg.d()?ytcfg.d()[k]:o},set:function(){var a=arguments;if(a.length>1)ytcfg.d()[a[0]]=a[1];else{var k;for(k in a[0])ytcfg.d()[k]=a[0][k]}}};
window.ytcfg.set('EME

In [52]:
title = result['videoDetails']['title']
views = result['videoDetails']['viewCount']
thumbnail_link = result['videoDetails']['thumbnail']['thumbnails'][-1]['url']


In [8]:
r = requests.get(link, headers={'User-Agent': ''})
likes = r.text[:r.text.find(' likes"')]
likes = likes[likes.rfind('"') +1:]

In [9]:
dislikes = r.text[:r.text.find(' dislikes"')]
dislikes = dislikes[dislikes.rfind('"') + 0:]
dislikes = dislikes.split(" ")
dislikes = dislikes[4]
dislikes = int(dislikes[:-2]) * 10
dislikes

3960

In [10]:
print("Title : " , title)
print("Views : ", int(views))
print("Thumbnail Link " , thumbnail_link)
print("Likes : ",int(likes.replace(",",'')))
print("Dislikes : ", int(dislikes))

Title :  Bulleya Full Video - ADHM|Ranbir, Aishwarya|Amit Mishra,Shilpa Rao|Pritam|Karan Johar
Views :  192647481
Thumbnail Link  https://i.ytimg.com/vi/wTgrZE9RWNY/maxresdefault.jpg
Likes :  1177317
Dislikes :  3960


In [11]:
def process_title(title):
    title = title.lower()
    stemmer = PorterStemmer()
    stopwords_english = STOP_WORDS 
    # remove hashtags
    # only removing the hash # sign from the word
    title = re.sub(r'#', '', title)
    # tokenize tweets
    eng_tokenize = English()
    title_doc =  eng_tokenize(title)
    
    token_list = []
    title_clean = []
    for token in title_doc:
        token_list.append(token.text)
        
    for word in token_list:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            title_clean.append(stem_word)

    title_clean = [*set(title_clean)] #Remove duplicate words
    return title_clean

In [12]:
title = process_title(title)

['adhm|ranbir',
 'aishwarya|amit',
 'johar',
 'video',
 'rao|pritam|karan',
 'shilpa',
 'bulleya',
 'mishra']

In [18]:
def mean_embed(tokens,model):
    vector_list = []
    
    for x in tokens:
        if x in model.wv:
            val = model.wv[x]
            vector_list.append(val)
            
    return np.mean(np.array(vector_list), axis=0)

In [19]:
print(mean_embed(title,ftt_model))

[-0.02467082  0.00813249 -0.02341757  0.01669894  0.01423362 -0.00557374
 -0.00077401  0.0162895   0.02669719 -0.0023495  -0.0224077  -0.02536016
 -0.00649614  0.05380472  0.00654594  0.02108519  0.00427737  0.00650466
 -0.02444075 -0.07186811 -0.05595468 -0.01131605 -0.02218621  0.00226638
  0.01313134  0.01592722 -0.00228781 -0.00899708  0.02977662 -0.00846096
  0.00581688 -0.04076356  0.03377625 -0.01852166 -0.00481232  0.00666781
 -0.01728875  0.01478392 -0.01651808 -0.00137147  0.03100106 -0.05869915
  0.01047401 -0.0219783  -0.04860753 -0.00948877  0.0144431  -0.02331432
 -0.05146639  0.03132464 -0.02109131 -0.00370781  0.02432416  0.02259669
 -0.01584108 -0.02862664  0.00175313  0.02234473  0.00099939 -0.01519996
  0.04094063 -0.04028839 -0.03249191  0.02778199  0.02927508  0.03233558
 -0.00770375 -0.01045178 -0.01142907  0.02346797  0.01725168 -0.00139014
 -0.00281048 -0.0090319   0.03327176 -0.00847802  0.03220086 -0.01590012
 -0.0077774   0.03146286 -0.02428405  0.00235064  0

In [ ]:
import streamlit